<a href="https://colab.research.google.com/github/AvivGelfand/Crypto-Data-Analysis-with-Python/blob/main/Crypto_EDA_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Crypto Data Analysis with Python - Aviv Gelfand

##Introduction
Binance is a cryptocurrency exchange which is the largest exchange in the world in terms of daily trading volume of cryptocurrencies.

In this project I will preform EDA on cryptocurrency exchange data that I will recieve from Binance's API.

Hope you enjoy it!

# Project Status
I learnd how to pull data from the my personal binance API (limited for data requests only).

I succeed to create a dataframe of transactions for Bitcoin to USDT. Now I can do the same for all kinds of data and build a function for that.

I will continue exploring the different values the df's include and will try to think of a specific diraction to explore. 

Any suggestion would be appriciated!

#Setting up libraries and accessing API

In [2]:
api_key = 'rCi6a53y9MJdIKPEfVXzrmt9tElZ5WniYAUjtEEgd8j5tVnsS85qzX63ut5A1yxI'
secret = 'WCEZUeUy6hJ5CVgjNUjSU03RzwKgMQeVhecvQsnb74jAQOQltffaBKaUZgoAUzpb'

In [3]:
!pip install python-binance 
#I'll keep in mind that later on it's possible to install: mplfinance for financial graphs

     |████████████████████████████████| 65 kB 2.4 MB/s 
     |████████████████████████████████| 288 kB 11.2 MB/s 
     |████████████████████████████████| 1.1 MB 65.1 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 
     |████████████████████████████████| 111 kB 45.7 MB/s 
     |████████████████████████████████| 144 kB 59.5 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 59.5 MB/s 


In [4]:
import pandas as pd
import numpy as np
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager

In [5]:
client = Client(api_key, secret)

#Data Preparing and processing for Analysis

## Loading the data set
I chose to load the data regarding BID and ASK of Bitcoin to USDT (USDT is the main coin that represents the US dolar).


In [6]:
historical_data_YTD = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_1DAY, '1 Jan 2021')
len(historical_data_YTD)

466

In [13]:
hist_df = pd.DataFrame(historical_data_YTD)
hist_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1609459200000,28923.63000000,29600.00000000,28624.57000000,29331.69000000,54182.92501100,1609545599999,1582526989.16187265,1314910,27455.80172500,802247744.54510409,0
1,1609545600000,29331.70000000,33300.00000000,28946.53000000,32178.33000000,129993.87336200,1609631999999,4073842163.67154117,2245922,67446.30524600,2110334723.88714587,0
2,1609632000000,32176.45000000,34778.11000000,31962.99000000,33000.05000000,120957.56675000,1609718399999,4057598425.49201649,2369698,59750.33287100,2004428433.93184622,0


##Adding names to columns

In [14]:
column_names = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                    'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
hist_df.columns = column_names
hist_df.head(3)

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,1609459200000,28923.63000000,29600.00000000,28624.57000000,29331.69000000,54182.92501100,1609545599999,1582526989.16187265,1314910,27455.80172500,802247744.54510409,0
1,1609545600000,29331.70000000,33300.00000000,28946.53000000,32178.33000000,129993.87336200,1609631999999,4073842163.67154117,2245922,67446.30524600,2110334723.88714587,0
2,1609632000000,32176.45000000,34778.11000000,31962.99000000,33000.05000000,120957.56675000,1609718399999,4057598425.49201649,2369698,59750.33287100,2004428433.93184622,0


**Quote asset volume** it is the volume in the second part of the pair. I.E. BTC/USDT - quote would be volume in USDT

## Setting the dates

In [15]:
hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')
hist_df.head(2)

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,2021-01-01,28923.63000000,29600.00000000,28624.57000000,29331.69000000,54182.92501100,2021-01-01 23:59:59.999000064,1582526989.16187265,1314910,27455.80172500,802247744.54510409,0
1,2021-01-02,29331.70000000,33300.00000000,28946.53000000,32178.33000000,129993.87336200,2021-01-02 23:59:59.999000064,4073842163.67154117,2245922,67446.30524600,2110334723.88714587,0


In [22]:
hist_df['Open'] = pd.to_numeric(hist_df['Open'], errors='coerce')
hist_df['High'] = pd.to_numeric(hist_df['High'], errors='coerce')
hist_df['Low'] = pd.to_numeric(hist_df['Low'], errors='coerce')
hist_df['Close'] = pd.to_numeric(hist_df['Close'], errors='coerce')
hist_df['Volume'] = pd.to_numeric(hist_df['Volume'], errors='coerce')
hist_df['Quote Asset Volume'] = pd.to_numeric(hist_df['Quote Asset Volume'], errors='coerce')
hist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Open Time           466 non-null    datetime64[ns]
 1   Open                466 non-null    float64       
 2   High                466 non-null    float64       
 3   Low                 466 non-null    float64       
 4   Close               466 non-null    float64       
 5   Volume              466 non-null    float64       
 6   Close Time          466 non-null    datetime64[ns]
 7   Quote Asset Volume  466 non-null    float64       
 8   Number of Trades    466 non-null    int64         
 9   TB Base Volume      466 non-null    object        
 10  TB Quote Volume     466 non-null    object        
 11  Ignore              466 non-null    object        
dtypes: datetime64[ns](2), float64(6), int64(1), object(3)
memory usage: 43.8+ KB


In [23]:
hist_df.describe()

,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades
count,466.000000,466.000000,466.000000,466.000000,466.000000,4.660000e+02,4.660000e+02
mean,46113.152017,47465.983798,44564.134099,46141.963605,64285.366515,2.869157e+09,1.685330e+06
std,9115.320777,9170.059154,9029.447118,9082.307174,36581.388855,1.451661e+09,6.996059e+05
min,28923.630000,29600.000000,28130.000000,29331.690000,10885.099220,4.581004e+08,2.637850e+05
25%,38482.672500,39846.175000,37158.750000,38571.132500,40860.715808,1.845521e+09,1.166077e+06
50%,45821.500000,47051.500000,44119.535000,45821.505000,54229.508258,2.602568e+09,1.585970e+06
75%,53769.845000,55481.750000,51153.282500,53770.547500,80031.039525,3.545868e+09,2.073595e+06
max,67525.820000,69000.000000,66222.400000,67525.830000,354347.243161,1.347769e+10,6.331062e+06


# What comes next?
All of my previous actions with loading and preparing the df will be defined in a function that I will use for more coins:
*   BTC-ETH
*   ETH-USDT
*   Other Coins?

Would be intersting to:


*   Analyse "Number of Trades" for each coin and see if there is a correlation
*   To check if there is an [Arbitrage](https://www.investopedia.com/terms/a/arbitrage.asp) from bitcoin to etherium to usdt.
*   Preform some visualisations trying to find other corralations.